# ID + Photo Attendance Tracker



In [ ]:
from tkinter import *
from tkinter import messagebox
import cv2
import os
import datetime
import csv
import webbrowser

# Create necessary directories
if not os.path.exists("student_photos"):
    os.makedirs("student_photos")
if not os.path.exists("attendance.csv"):
    with open("attendance.csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["ID", "Name", "Date", "Time"])

class System:
    def __init__(self, root):
        self.root = root
        self.root.title("Automate Attendance System")
        self.root.geometry("700x550+300+50")
        self.root.resizable(False, False)

        title = Label(self.root, text='Student Attendance System', bg='darkblue', fg='white', font=("Arial", 16, "bold"))
        title.pack(fill=X)

        frame1 = Frame(self.root, bd=2, relief=RIDGE, bg="lightyellow")
        frame1.place(x=10, y=40, width=330, height=450)

        frame2 = Frame(self.root, bd=2, relief=RIDGE, bg="lightyellow")
        frame2.place(x=360, y=40, width=330, height=450)

        # Registration section
        Label(frame1, text='Registration', bg='darkred', fg='white', font=("Arial", 14, "bold")).place(x=2, y=2, width=325)

        Label(frame1, text='ID:', bg='lightyellow', fg='darkred', font=("Arial", 12)).place(x=2, y=50)
        self.entry_id = Entry(frame1, bg='white', fg='black', font=("Arial", 12))
        self.entry_id.place(x=60, y=50, width=200)
        self.entry_id.insert(0, self.get_next_id())

        Label(frame1, text='Name:', bg='lightyellow', fg='darkred', font=("Arial", 12)).place(x=2, y=100)
        self.entry_name = Entry(frame1, bg='white', fg='black', font=("Arial", 12))
        self.entry_name.place(x=60, y=100, width=200)

        Button(frame1, text='Take Photo', command=self.take_photo, bg='lightblue', font=("Arial", 12)).place(x=90, y=150)

        Button(frame1, text='View Saved Photos', command=self.view_photos, bg='lightgreen', font=("Arial", 12)).place(x=60, y=200)

        Button(frame1, text='Mark Attendance', command=self.mark_attendance_manual, bg='lightblue', font=("Arial", 12)).place(x=80, y=260)

        # Attendance section
        Label(frame2, text='Attendance', bg='darkred', fg='white', font=("Arial", 14, "bold")).place(x=2, y=2, width=325)

        Button(frame2, text='Show Today\'s Attendance', command=self.show_today_attendance, bg='lightgreen', font=("Arial", 12)).place(x=30, y=60, width=270)

        self.attendance_output = Text(frame2, height=18, width=39)
        self.attendance_output.place(x=15, y=110)

    def get_next_id(self):
        existing_ids = []
        for file in os.listdir("student_photos"):
            if "_" in file:
                id_part = file.split("_")[0]
                if id_part.isdigit():
                    existing_ids.append(int(id_part))
        return str(max(existing_ids, default=0) + 1)

    def take_photo(self):
        sid = self.entry_id.get()
        name = self.entry_name.get()

        if not sid or not name:
            messagebox.showerror("Error", "Please enter both ID and Name")
            return

        filename = f"student_photos/{sid}_{name}.jpg"
        if os.path.exists(filename):
            messagebox.showwarning("Duplicate", "Photo already exists for this student.")
            return

        cap = cv2.VideoCapture(0)
        messagebox.showinfo("Info", "Press 's' to capture photo, 'q' to cancel.")
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            cv2.imshow("Capture Photo - Press 's' to save", frame)
            key = cv2.waitKey(1)
            if key == ord('s'):
                cv2.imwrite(filename, frame)
                messagebox.showinfo("Success", f"Photo saved as {filename}")
                break
            elif key == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        self.entry_id.delete(0, END)
        self.entry_id.insert(0, self.get_next_id())
        self.entry_name.delete(0, END)

    def view_photos(self):
        webbrowser.open(os.path.abspath("student_photos"))

    def mark_attendance_manual(self):
        sid = self.entry_id.get()
        name = self.entry_name.get()

        if not sid or not name:
            messagebox.showerror("Error", "Please enter both ID and Name")
            return

        now = datetime.datetime.now()
        date = now.strftime("%Y-%m-%d")
        time = now.strftime("%H:%M:%S")
        with open("attendance.csv", "a", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([sid, name, date, time])
        messagebox.showinfo("Success", "Attendance marked manually.")

        self.entry_id.delete(0, END)
        self.entry_id.insert(0, self.get_next_id())
        self.entry_name.delete(0, END)

    def show_today_attendance(self):
        today = datetime.datetime.now().strftime("%Y-%m-%d")
        self.attendance_output.delete(1.0, END)
        with open("attendance.csv", "r") as f:
            reader = csv.DictReader(f)
            found = False
            for row in reader:
                if row["Date"] == today:
                    self.attendance_output.insert(END, f"{row['ID']} - {row['Name']} at {row['Time']}\n")
                    found = True
            if not found:
                self.attendance_output.insert(END, "No attendance found for today.")

root = Tk()
obj = System(root)
root.mainloop()
